In [ ]:
import datetime

import numpy as np
from tqdm import tqdm
from environment import sun_position, moon_position
import matplotlib.pyplot as plt

from utils import PiecewiseConstant
time = datetime.datetime.now(datetime.timezone.utc)
import pandas as pd

s = []
m = []
t = []

time = pd.date_range(time, time+ datetime.timedelta(hours=2), freq=datetime.timedelta(seconds=0.25))

def floor_time_to_minute(t: datetime.datetime) -> datetime.datetime:
    return t.replace(second=0, microsecond=0)

self.sun_position = PiecewiseConstant( fn=sun_position, time_bucket_fn=floor_time_to_minute)

my_moon_position = PiecewiseConstant(
    fn=moon_position,
    time_bucket_fn=floor_time_to_minute,
)


with tqdm(total=time.size) as pbar:
    for i in time:
        s.append(my_sun_position(i))
        m.append(my_moon_position(i))

        pbar.update()


In [51]:
import datetime
from scipy.spatial.transform import Rotation as R
import numpy as np
from astropy.coordinates import EarthLocation
import astropy.units as u
from astropy.coordinates import ITRS, GCRS
from astropy.time import Time
import numpy as np

def ecef_to_eci_matrix(t: datetime.datetime) -> R:
    t_ast = Time(t)

    itrs = ITRS(obstime=t_ast)
    gcrs = GCRS(obstime=t_ast)

    # This is the expensive part — but done once
    return R.from_matrix(itrs.transform_to(gcrs).get_itrs_to_gcrs_matrix())

ecef_to_eci_matrix(time[0])

ValueError: Cannot transform a frame with no data

In [ ]:
# old 3 min, 160 it/s
# new
import pandas as pd
# plt.plot((time-time[0]).total_seconds()/60, (np.array(m)[:, 0]))
# plt.plot((time-time[0]).total_seconds()/3600, (1- np.array(s)[:, :]/np.array(s)[0, :][np.newaxis, :])*100)
# plt.plot((time-time[0]).total_seconds()/60, (1- np.array(m)[:, :]/np.array(m)[0, :][np.newaxis, :])*100)
plt.grid()

In [ ]:
from filterpy.kalman import ExtendedKalmanFilter
import casadi as ca

# TODO: implement extended kalman filter wrapper
class IMUEKF(ExtendedKalmanFilter):
    def __init__(self, dim_x, dim_z, f: ca.Function, g: ca.Function):
        super().__init__(dim_x, dim_z)

        self.f = f # process model

        dt = ca.SX.sym("dt")
        self.F = integrator(self.f, self.x, u, dt) # function returns next state
        
        self.g = g # measurement model

        self.g_jac = g.jacobian()

    def predict_x(self, u, dt):
        self.x = self.F(self.x, u, dt)

    def update(self, z):

        super().update(z, self.g_jac, self.g)


In [ ]:
from io import StringIO
import sys
from IPython.display import Latex
import casadi as ca
import re
import numpy as np


def latex_print(expr, sub=False):
    old_stdout = sys.stdout
    sys.stdout = mystdout = StringIO()
    expr.print_sparse()
    sys.stdout = old_stdout
    x = mystdout.getvalue().split('\n')

    shape_split = x[0].split(':')[1].split(',')[0].strip().split('-')
    m = int(shape_split[0])
    n = int(shape_split[2])
    e = {}
    b = {
        '*': r' \cdot ',
        'cos': r'\cos',
        'sin': r'\sin'
    }
    for l in ['alpha', 'beta', 'gamma', 'Gamma',
            'delta', 'Delta', 'epsilon', 'zeta', 'eta', 'theta',
            'Theta', 'iota', 'kappa', 'lambda', 'Lambda', 'mu',
            'nu', 'xi', 'pi', 'rho', 'sigma', 'tau', 'upsilon',
            'phi', 'Phi', 'chi', 'psi', 'Psi', 'omega', 'Omega']:
        b[l] = '\\' + l
    s = r'\b(' + '|'.join(sorted(b.keys(), key=len, reverse=True)) + r')\b'
    # fix for escaped keys
    s = s.replace('*', '\\*')
    b_pattern = re.compile(s)
    t = {}
    for i in range(1, len(x)):
        line = x[i].strip()
        if line[0] == "@":
            split = line.split("=")
            term = split[1].replace(',', '')
            term = b_pattern.sub(lambda x: b[x.group()], term)
            e[split[0]] = term
        elif line[0] == "(":
            split = line.split("->")
            indices = split[0].strip().replace('(', '').replace(')', '').split(',')
            i = int(indices[0].strip())
            j = int(indices[1].strip())
            t[(i, j)] = str(split[1])
    s = r"\begin{bmatrix}"
    for i in range(m):
        row = []
        for j in range(n):
            term = "0";
            if (i, j) in t.keys():
                term = t[(i, j)]
            term = b_pattern.sub(lambda x: b[x.group()], term)
            if sub:
                for k in sorted(e.keys(), key=len, reverse=True):
                    term = term.replace(k, e[k])
            row += [r"{:s}".format(term)]
        s += "&".join(row) + r"\\"
    s += r"\end{bmatrix}"
    if not sub:
        s += r"\begin{align}"
        for k in e.keys():
            s += r"{:s}: && {:s} \\".format(k, e[k])
        s += r"\end{align}"
    return s

def latex_display(x, sub=True):
    return Latex(latex_print(x, sub))


# symbolic dynamics model for estimation and control
q_BI = ca.SX.sym('q', 4)
omega = ca.SX.sym(r'\omega', 3)
omega_w = ca.SX.sym(r"\Omega", 3)
x = ca.vertcat(q_BI, omega, omega_w)
u = ca.SX.sym('u', 6)
u_rw = u[:3]
u_mag = u[3:]

B_B = ca.SX.sym('B', 3) # where B is the measured magnetic field from the                   
A_hat = np.eye(3)
J_hat = np.eye(3)
J_w = np.ones(3)
K_t_dash = 1
K_e_rw = 1
K_mag = 1

# actuators
h_int = A_hat @ (J_w * (omega_w + A_hat @ omega)) 
tau_rw = K_t_dash * (A_hat @ (u_rw - K_e_rw * omega_w)) 

# r_norm = ca.norm_2(r_ECI) # instead of dipole model the measurements are used
# r_hat = r_ECI / r_norm
# B_I = (3 * r_hat * (ca.dot(M_dipole, r_hat)) - M_dipole) / r_norm**3
tau_mag = ca.cross(K_mag * u_mag, B_B) 

# attitude kinematics
q_dot = ca.SX.sym(r"\dot{q}", 4)
q_dot[:3] = 0.5 * (omega * q_BI[3] + ca.cross(omega, q_BI[:3]))
q_dot[3] = -0.5 * ca.dot(omega, q_BI[:3])

# attitude dynamics
cross_term = ca.cross(omega, J_hat @ omega + h_int)
total_torque = tau_mag - tau_rw - cross_term
omega_dot = ca.solve(J_hat, total_torque)
# omega_w_dot = u_rw / J_w - A_hat @ omega_dot


dx = ca.vertcat(q_dot, omega_dot)
f = ca.Function("f", [x, u, B_B], [dx], ["x", "u", "B"], ["dx"])

latex_display(dx)

In [ ]:

f_jac = f.jacobian()

# x = 3 * np.ones(10)

# u = 2 * np.ones(6)

# B_B = np.ones(3)

# f_jac([x, u, B_B])

out = np.zeros(7)

A, B, _ = f_jac(x, u, B_B, out)

A